In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_facenet import FaceNet
from numpy.linalg import norm
detector = MTCNN() #Initialize MTCNN detector for face detection
embedder = FaceNet() # Initialize FaceNet embedder for facial recognition

In [ ]:
# Defining function to provide embedding of face of an individual
def img_to_embedding(image_path):
    img=cv2.imread(image_path)
    img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces=detector.detect_faces(img_rgb)[0]
        
    x,y,w,h=faces['box']
    x1,y1=abs(x),abs(y)
    fi=img_rgb[y:y+h,x:x+w]
    face=cv2.resize(fi,(160,160))
    face = np.expand_dims(face, axis=0)
    embedding = embedder.embeddings(face)
    return embedding[0]

In [ ]:
# creating database to store the name as key and respective facial embeddings as value.
database = {} 

database["agastya"] = img_to_embedding("database/agastya.jpeg")
database["anu_son"] = img_to_embedding("database/anu_son.jpeg")
database["raj"] = img_to_embedding("database/raj.jpeg")
database["resamia"] = img_to_embedding("database/ris3.jpg")
database["tingu"] = img_to_embedding("database/pikachu.jpg")
database["ash"] = img_to_embedding("database/ash3.jpg")
database["mittal"] = img_to_embedding("database/ayush mittal.jpg")
database["alok"] = img_to_embedding("database/alok.jpg")
database["nishant"] = img_to_embedding("database/nishant.jpg")

In [ ]:
# Defining function to provide embeddings of faces detected in an image
def multifaces_embedding(img_path):
    img=cv2.imread(img_path)
    img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces=detector.detect_faces(img_rgb) # Detected every faces in an image
    embeddings=[]
    boxes=[]
    for face in faces:
        x,y,w,h=face['box']
        x,y=abs(x),abs(y)
        fi=img_rgb[y:y+h,x:x+w]
        fi=cv2.resize(fi, (160, 160))
        fi = np.expand_dims(fi, axis=0)
        embedding = embedder.embeddings(fi) # Finding embedding of detected face
        embeddings.append(embedding[0])
        boxes.append((x, y, w, h))
    return embeddings,boxes,img_rgb

In [ ]:
# Created this function to draw boxes around the faces and also shows names of recognised individuals.
def draw_boxes(image_rgb, recognized_boxes):
    for name, (x, y, width, height) in recognized_boxes:
        cv2.rectangle(image_rgb, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(image_rgb, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image_rgb

In [ ]:
# using cosine similarity to find simalarities between two embeddings.
def cosine_similarity(embedding1, embedding2):
    a_b = np.dot(embedding1, embedding2)
    a = norm(embedding1)
    b= norm(embedding2)
    return a_b / (a * b)


In [ ]:
# Defining a function to recognize face from all the images of test directory
def Recognize_Faces(test_dir, database, threshold=0.7):
    for filename in os.listdir(test_dir):
        photo_path = os.path.join(test_dir, filename)
        
        embeddings, boxes, image_rgb = multifaces_embedding(photo_path)
        
        recognized_boxes = []
        
        for embedding, box in zip(embeddings, boxes):
            for name, db_embedding in database.items():
                if cosine_similarity(embedding, db_embedding)>threshold:
                    recognized_boxes.append((name, box))
                    break

        image_with_boxes = draw_boxes(image_rgb, recognized_boxes)
        
        # Display the image
        plt.figure(figsize=(10, 8), dpi=500)
        plt.imshow(image_with_boxes)
        plt.axis('off')
        plt.show()
        



In [ ]:
Recognize_Faces('test1', database)